In [2]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import json
import numpy as np

# open files
df_hdiy = pd.read_csv('../data/cleaned_hdi_py_data.csv')

In [11]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Assume df_hdiy is already defined and contains the necessary data

years = list(range(1990, 2023))
hdi_columns = [f'hdi_{year}' for year in years]
df_hdiyn = df_hdiy[['country'] + hdi_columns]

# Melt the DataFrame to long format for easier plotting
df_melted = df_hdiyn.melt(id_vars=['country'], var_name='year', value_name='hdi')

# Convert year to numeric
df_melted['year'] = df_melted['year'].str.extract('(\d+)').astype(int)

# Define bins and labels for HDI categories
bins = [0, 0.55, 0.69, 1.0]
labels = ['Laag (< 0.55)', 'Midden (0.55 - 0.69)', 'Hoog (0.69 <)']

# Bin the HDI values
df_melted['category'] = pd.cut(df_melted['hdi'], bins=bins, labels=labels, include_lowest=True)

# Group by year and category
df_grouped = df_melted.groupby(['year', 'category'], observed=True).size().unstack(fill_value=0).reset_index()

# Define colors for the categories
colors = {
    'Laag (< 0.55)': 'viridis_2',
    'Midden (0.55 - 0.69)': 'orange',
    'Hoog (0.69 <)': 'green'
}

# Create the plot
fig = go.Figure()

# Add traces for each category with specified colors
for category in labels:
    fig.add_trace(go.Bar(
        x=df_grouped['year'],
        y=df_grouped[category],
        name=category,
        marker_color=colors[category]
    ))

# Update layout
fig.update_layout(
    barmode='stack',
    title='Aantal landen per HDI Categorie (1990 - 2022)',
    xaxis=dict(title='Jaartal'),
    yaxis=dict(title='Aantal landen'),
    legend=dict(title='HDI Categorie', x=1.05, y=1),
    font=dict(size=18),
    autosize=False
)

fig.show()


ValueError: 
    Invalid value of type 'builtins.str' received for the 'color' property of bar.marker
        Received value: 'viridis_2'

    The 'color' property is a color and may be specified as:
      - A hex string (e.g. '#ff0000')
      - An rgb/rgba string (e.g. 'rgb(255,0,0)')
      - An hsl/hsla string (e.g. 'hsl(0,100%,50%)')
      - An hsv/hsva string (e.g. 'hsv(0,100%,100%)')
      - A named CSS color:
            aliceblue, antiquewhite, aqua, aquamarine, azure,
            beige, bisque, black, blanchedalmond, blue,
            blueviolet, brown, burlywood, cadetblue,
            chartreuse, chocolate, coral, cornflowerblue,
            cornsilk, crimson, cyan, darkblue, darkcyan,
            darkgoldenrod, darkgray, darkgrey, darkgreen,
            darkkhaki, darkmagenta, darkolivegreen, darkorange,
            darkorchid, darkred, darksalmon, darkseagreen,
            darkslateblue, darkslategray, darkslategrey,
            darkturquoise, darkviolet, deeppink, deepskyb